In [ ]:
import app

In [34]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

True

In [35]:
def get_commit_changes(owner, repo, commit_hash, github_token=None):
    """
    특정 커밋에서 변경된 모든 파일의 diff 내용을 가져옵니다.
    """
    # 특정 커밋의 상세 정보를 가져오는 API URL
    api_url = f"https://api.github.com/repos/{owner}/{repo}/commits/{commit_hash}"
    headers = {"Accept": "application/vnd.github.v3+json"}
    if github_token:
        headers["Authorization"] = f"token {github_token}"

    print(f"API 요청 URL: {api_url}")
    
    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()  # 요청 실패 시 예외 발생
        commit_data = response.json()

        # 변경된 파일 정보가 없으면 빈 리스트 반환
        if 'files' not in commit_data:
            return []

        # LLM에게 전달할 정보를 가공
        changes = []
        for file_info in commit_data['files']:
            # 'patch' 필드가 없는 경우 (예: 파일명만 변경)는 건너뜀
            if 'patch' not in file_info:
                continue

            change_detail = {
                'filename': file_info['filename'], # file path and file name
                'status': file_info['status'],     # 'added', 'modified', 'removed' 등
                'patch': file_info['patch']        # 변경된 내용 (diff)
            }
            changes.append(change_detail)
            
        return changes

    except requests.exceptions.RequestException as e:
        print(f"❌ GitHub에서 커밋 정보를 가져오는 데 실패했습니다: {e}")
        if 'response' in locals():
            print(f"error contents: {response.text}")
        return None



In [ ]:
if __name__ == "__main__":
    # --- 💡 USER INPUT : user infomation---
    REPO_OWNER = "minmings111" # user ID
    REPO_NAME = "AICA_study" # user's repository name
    
    # 2개 이상의 파일이 수정된 커밋 해시
    # USER INPUT
    TARGET_COMMIT_HASH = "fdedda63de2f4ccae4066d95fa0b190c77121967" 
    
    # get GITHUB_API_TOKEN
    # ithub_token get from streamlit.ipynb
    GITHUB_API_TOKEN = os.getenv(github_token)

    print(f"'{REPO_OWNER}/{REPO_NAME}' 리포지토리의")
    print(f"COMMIT '{TARGET_COMMIT_HASH[:7]}...' 의 변경 내역을 가져옵니다.")
    print("-" * 30)

    commit_changes = get_commit_changes(
        owner=REPO_OWNER,
        repo=REPO_NAME,
        commit_hash=TARGET_COMMIT_HASH,
        github_token=GITHUB_API_TOKEN
    )

    if commit_changes:
        print(f"\n✅ 총 {len(commit_changes)}개의 파일에서 변경 내역을 발견했습니다!")
        print("-" * 30)
        
        # LLM에게 이 'commit_changes' 전체를 넘겨주면 됩니다.
        for change in commit_changes:
            print(f"📄 파일명: {change['filename']} ({change['status']})")
            print("🎨 변경 내용 (Patch):")
            print(change['patch'])
            print("-" * 20)

'minmings111/AICA_study' 리포지토리의
COMMIT 'fdedda6...' 의 변경 내역을 가져옵니다.
------------------------------
API 요청 URL: https://api.github.com/repos/minmings111/AICA_study/commits/fdedda63de2f4ccae4066d95fa0b190c77121967

✅ 총 23개의 파일에서 변경 내역을 발견했습니다!
------------------------------
📄 파일명: labs_java/workspace/springBtWeb/src/main/java/app/labs/HomeController.java (added)
🎨 변경 내용 (Patch):
@@ -0,0 +1,85 @@
+package app.labs;
+
+
+
+
+import java.time.LocalDateTime;
+import java.time.format.DateTimeFormatter;
+import java.util.List;
+import java.util.Map;
+
+import org.springframework.beans.factory.annotation.Autowired;
+import org.springframework.stereotype.Controller;
+import org.springframework.ui.Model;
+import org.springframework.web.bind.annotation.GetMapping;
+import org.springframework.web.bind.annotation.PostMapping;
+import org.springframework.web.bind.annotation.RequestMapping;
+import org.springframework.web.bind.annotation.ResponseBody;
+
+import app.labs.ex01.TemplateBasicService;
+imp